This notebook demonstrates how to use [OptKeras](
https://github.com/Minyus/optkeras
), a Python wrapper around Keras and Optuna



##  Set up Google Colab environment

To run in Google Colab, specify a directory in Google Drive. (GPU is recommended.)

To run in an environment other than Google Colab, just skip this code.

In [1]:
""" To use Google Drive with Colab, 
1. set use_google_drive to True, and
2. specify a directory in Google Drive (Modify as in your Google Drive)
(You will need to authorize manually.)
"""
use_google_drive = False
workdir = '/content/drive/My Drive/Colab Notebooks/OptKeras_Example_Output'


import os

try:
    if use_google_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        # Create target directory & all intermediate directories if don't exists
        if not os.path.exists(workdir):
            os.makedirs(workdir)
            print('## Directory: ' , workdir ,  ' was created.') 
        os.chdir(workdir)
        print('## Current working directory: ', os.getcwd())
except:
    print('Run the code without using Google Drive.')
        
try:    
    print('## Check the uptime. (Google Colab reboots every 12 hours)')
    !cat /proc/uptime | awk '{print "Uptime is " $1 /60 /60 " hours (" $1 " sec)"}'
    print('## Check the GPU info')
    !nvidia-smi
    print('## Check the OS') 
    !cat /etc/issue
    print('## Check the Python version') 
    !python --version
    print('## Check the memory')
    !free -h
    print('## Check the disk')
    !df -h
except:
    print('Run the code assuming the environment is not Google Colab.')

## Check the uptime. (Google Colab reboots every 12 hours)
Uptime is 0.0177 hours (63.72 sec)
## Check the GPU info
Fri Mar  8 06:09:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+---------------------------------------------------

## Install Optuna 0.7.0

In [2]:
!pip install optuna==0.7.0

    100% |████████████████████████████████| 61kB 2.9MB/s 
    100% |████████████████████████████████| 81kB 7.2MB/s 
    100% |████████████████████████████████| 92kB 25.6MB/s 
    100% |████████████████████████████████| 51kB 17.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/8c/97/c7/868e9cd5d6a23cfb98266f30d59131e5bd441132bdf8da18d5
  Stored in directory: /root/.cache/pip/wheels/92/f0/ac/2ba2972034e98971c3654ece337ac61e546bdeb34ca960dc8c
Successfully built optuna pyperclip


## Install OptKeras 0.0.1

In [3]:
!pip install optkeras==0.0.1
# Alternatively you can install from the GitHub repository
#!pip install git+https://github.com/Minyus/optkeras.git

## Import modules

In [4]:
import numpy as np
import pandas as pd

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
import keras.backend as K

import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

# (Optional) Disable messages from Optuna below WARN level.
optuna.logging.set_verbosity(optuna.logging.WARN) 

Using TensorFlow backend.


Keras 2.2.4
TensorFlow 1.13.1
Optuna 0.7.0
OptKeras 0.0.1


## Set up Dataset

In [5]:
dataset_name = 'MNIST'

if dataset_name in ['MNIST']:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    img_x, img_y = x_train.shape[1], x_train.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1)
    x_test = x_test.reshape(-1, img_x, img_y, 1)   
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    num_classes = 10
    input_shape = (img_x, img_y, 1)

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
print('x_train: ', x_train.shape)
print('y_train', y_train.shape)
print('x_test: ', x_test.shape)
print('y_test', y_test.shape)
print('input_shape: ', input_shape )    

x_train:  (60000, 28, 28, 1)
y_train (60000,)
x_test:  (10000, 28, 28, 1)
y_test (10000,)
input_shape:  (28, 28, 1)


## A simple Keras model

In [7]:
model = Sequential()
model.add(Conv2D(
    filters = 32, 
    kernel_size = 3, 
    strides = 1,
    activation = 'relu', 
    input_shape = input_shape ))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(optimizer = Adam(), 
            loss='sparse_categorical_crossentropy', metrics=['accuracy'])			  
model.fit(x_train, y_train, 
          validation_data = (x_test, y_test), shuffle = True,
          batch_size = 512, epochs = 2) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 17s 283us/step - loss: 0.4527 - acc: 0.8834 - val_loss: 0.2093 - val_acc: 0.9416
Epoch 2/2
60000/60000 [==============================] - 2s 41us/step - loss: 0.1689 - acc: 0.9532 - val_loss: 0.1291 - val_acc: 0.9645


## Optimization of a simple Keras model without pruning

In [8]:
study_name = dataset_name + '_Simple'

""" Step 1. Instantiate OptKeras class
You can specify arguments for Optuna's create_study method and other arguments 
for OptKeras such as enable_pruning. 
"""

ok = OptKeras(study_name=study_name)


""" Step 2. Define objective function for Optuna """

def objective(trial):
    
    """ Clear the backend (TensorFlow). See:
    https://www.tensorflow.org/api_docs/python/tf/keras/backend/clear_session
    """
    K.clear_session() 
    
    """ Step 2.1. Define parameters to try using methods of optuna.trial such as 
    suggest_categorical. In this simple demo, try 2*2*2*2 = 16 parameter sets: 
    2 values specified in list for each of 4 parameters 
    (filters, kernel_size, strides, and activation for convolution).
    """    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    """ Step 2.2. Specify callbacks(trial) and keras_verbose in fit 
    (or fit_generator) method of Keras model
    """
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    """ Step 2.3. Return trial_best_value (or latest_value) """
    return ok.trial_best_value

""" Step 3. Run optimize. 
Set n_trials and/or timeout (in sec) for optimization by Optuna
"""
ok.optimize(objective, timeout = 3*60) # Run for 3 minutes for demo

[2019-03-08 06:10:30.518291] Ready for optimization. (message printed as verbose is set to 1+)
[2019-03-08 06:10:30.839458] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-03-08 06:10:35.734725] Latest trial id: 0, value: 0.07829999923706055 (TrialState.COMPLETE) | Best trial id: 0, value: 0.07829999923706055, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'linear'}
[2019-03-08 06:10:38.087010] Latest trial id: 1, value: 0.05369999933242797 (TrialState.COMPLETE) | Best trial id: 1, value: 0.05369999933242797, parameters: {'filters': 32, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-03-08 06:10:40.983369] Latest trial id: 2, value: 0.043499999332427985 (TrialState.COMPLETE) | Best trial id: 2, value: 0.043499999332427985, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'relu'}
[2019-03-08 06:10:44.053877] Latest trial id: 3, value: 0.05949999952316287 (TrialState.COMPL

In [9]:
""" Show Results """
print('Best trial id: ', ok.best_trial.trial_id)
print('Best value:', ok.best_trial.value)
print('Best parameters: \n', ok.best_trial.params)

"""
Alternatively, you can access Optuna's study object to, for example, 
get the best parameters as well.
Please note that study.best_trial returns error if optimization trials 
were not completed (e.g. if you interupt execution) as of Optuna 0.7.0, 
so usage of OptKeras is recommended.
"""
print("Best parameters (retrieved directly from Optuna)", ok.study.best_trial.params)

""" Check the Optuna CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
display(pd.read_csv(ok.optuna_log_file_path))

""" Check the Keras CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
display(pd.read_csv(ok.keras_log_file_path))

Best trial id:  43
Best value: 0.020699999237060562
Best parameters: 
 {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Best parameters (retrieved directly from Optuna) {'filters': 64, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
Data Frame read from MNIST_Simple_Optuna.csv 



,trial_id,state,value,datetime_start,datetime_complete,activation,filters,kernel_size,strides,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,TrialState.COMPLETE,0.0783,2019-03-08 06:10:30.541164,2019-03-08 06:10:35.638037,linear,64,5,1,2019-03-08 06:10:33.567568,2019-03-08 06:10:35.611090,val_error,0,0.917650,0.082350,0.294218,0.9217,0.0783,0.275228
1,1,TrialState.COMPLETE,0.0537,2019-03-08 06:10:35.638591,2019-03-08 06:10:37.985077,relu,32,5,2,2019-03-08 06:10:37.190775,2019-03-08 06:10:37.963941,val_error,1,0.931783,0.068217,0.241850,0.9463,0.0537,0.192020
2,2,TrialState.COMPLETE,0.0435,2019-03-08 06:10:37.985647,2019-03-08 06:10:40.884892,relu,64,5,2,2019-03-08 06:10:39.755490,2019-03-08 06:10:40.861909,val_error,2,0.942917,0.057083,0.201087,0.9565,0.0435,0.155531
3,3,TrialState.COMPLETE,0.0595,2019-03-08 06:10:40.885549,2019-03-08 06:10:43.953312,relu,64,3,2,2019-03-08 06:10:42.754567,2019-03-08 06:10:43.929170,val_error,3,0.927867,0.072133,0.254514,0.9405,0.0595,0.208976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,43,TrialState.COMPLETE,0.0207,2019-03-08 06:13:13.326709,2019-03-08 06:13:19.114625,relu,64,5,1,2019-03-08 06:13:16.723648,2019-03-08 06:13:19.087320,val_error,43,0.973100,0.026900,0.099360,0.9793,0.0207,0.070524
44,44,TrialState.COMPLETE,0.0220,2019-03-08 06:13:19.123635,2019-03-08 06:13:24.681784,relu,64,5,1,2019-03-08 06:13:22.304918,2019-03-08 06:13:24.654582,val_error,44,0.970433,0.029567,0.105150,0.9780,0.0220,0.075203
45,45,TrialState.COMPLETE,0.0375,2019-03-08 06:13:24.689711,2019-03-08 06:13:27.644822,relu,64,5,2,2019-03-08 06:13:26.538335,2019-03-08 06:13:27.621304,val_error,45,0.946517,0.053483,0.191103,0.9625,0.0375,0.136838
46,46,TrialState.COMPLETE,0.0780,2019-03-08 06:13:27.652353,2019-03-08 06:13:33.064467,linear,64,3,1,2019-03-08 06:13:30.707785,2019-03-08 06:13:33.036852,val_error,46,0.918817,0.081183,0.289438,0.9220,0.0780,0.278177


Data Frame read from MNIST_Simple_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Monitor,_Trial_id,acc,error,loss,val_acc,val_error,val_loss
0,0,2019-03-08 06:10:31.113475,2019-03-08 06:10:33.484209,val_error,0,0.882033,0.117967,0.414477,0.9198,0.0802,0.287420
1,1,2019-03-08 06:10:33.567568,2019-03-08 06:10:35.611090,val_error,0,0.917650,0.082350,0.294218,0.9217,0.0783,0.275228
2,0,2019-03-08 06:10:36.158179,2019-03-08 06:10:37.132854,val_error,1,0.846967,0.153033,0.625918,0.9193,0.0807,0.281282
3,1,2019-03-08 06:10:37.190775,2019-03-08 06:10:37.963941,val_error,1,0.931783,0.068217,0.241850,0.9463,0.0537,0.192020
...,...,...,...,...,...,...,...,...,...,...,...
90,0,2019-03-08 06:13:25.123696,2019-03-08 06:13:26.452529,val_error,45,0.861417,0.138583,0.539497,0.9347,0.0653,0.231427
91,1,2019-03-08 06:13:26.538335,2019-03-08 06:13:27.621304,val_error,45,0.946517,0.053483,0.191103,0.9625,0.0375,0.136838
92,0,2019-03-08 06:13:28.064917,2019-03-08 06:13:30.633108,val_error,46,0.883833,0.116167,0.411419,0.9163,0.0837,0.287490
93,1,2019-03-08 06:13:30.707785,2019-03-08 06:13:33.036852,val_error,46,0.918817,0.081183,0.289438,0.9220,0.0780,0.278177


## Randomized Grid Search of a simple Keras model

In [10]:
study_name = dataset_name + '_GridSearch'

""" To run randomized grid search, set random_grid_search_mode True """
ok = OptKeras(study_name=study_name, random_grid_search_mode=True)

def objective(trial):
   
    K.clear_session()
    
    model = Sequential()
    model.add(Conv2D(
        filters = trial.suggest_categorical('filters', [32, 64]), 
        kernel_size = trial.suggest_categorical('kernel_size', [3, 5]), 
        strides = trial.suggest_categorical('strides', [1, 2]), 
        activation = trial.suggest_categorical('activation', ['relu', 'linear']), 
        input_shape = input_shape ))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer = Adam(), 
                loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(x_train, y_train, 
              validation_data = (x_test, y_test), shuffle = True,
              batch_size = 512, epochs = 2,
              callbacks = ok.callbacks(trial), 
              verbose = ok.keras_verbose )  
    
    return ok.trial_best_value

""" Set the number of parameter sets as n_trials for complete grid search """
ok.random_grid_search(objective, n_trials = 2*2*2*2) # 2*2*2*2 = 16 param sets

[2019-03-08 06:13:33.325220] Ready for optimization. (message printed as verbose is set to 1+)
[2019-03-08 06:13:33.757307] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-03-08 06:13:36.405106] Completed:   6% (    1 /    16)
[2019-03-08 06:13:36.514169] Latest trial id: 0, value: 0.07819999923706056 (TrialState.COMPLETE) | Best trial id: 0, value: 0.07819999923706056, parameters: {'filters': 64, 'kernel_size': 5, 'strides': 2, 'activation': 'linear'}
[2019-03-08 06:13:40.264285] Completed:  12% (    2 /    16)
[2019-03-08 06:13:40.383141] Latest trial id: 1, value: 0.025199999523162875 (TrialState.COMPLETE) | Best trial id: 1, value: 0.025199999523162875, parameters: {'filters': 32, 'kernel_size': 5, 'strides': 1, 'activation': 'relu'}
[2019-03-08 06:13:43.186837] Completed:  19% (    3 /    16)
[2019-03-08 06:13:43.300175] Latest trial id: 2, value: 0.08229999942779542 (TrialState.COMPLETE) | Best trial id: 1, value: 0.02519999952

## Optimization of a Keras model using more Optuna's features such as pruning

In [11]:
study_name = dataset_name + '_Optimized'

ok = OptKeras( 
    # parameters for optuna.create_study
    storage='sqlite:///' + study_name + '_Optuna.db', 
    sampler=optuna.samplers.TPESampler(
        consider_prior=True, prior_weight=1.0, 
        consider_magic_clip=True, consider_endpoints=False, 
        n_startup_trials=10, n_ei_candidates=24, 
        seed=None), 
    pruner=optuna.pruners.SuccessiveHalvingPruner(
        min_resource=1, reduction_factor=4, min_early_stopping_rate=0), 
    study_name = study_name, 
    direction='minimize', 
    load_if_exists = True,
    # parameters for OptKeras
    monitor='val_error', # Either 'val_error' (1 - val_acc) or 'val_loss'
    enable_pruning=True, 
    models_to_keep=1, # Either 1, 0, or -1 (save all models) 
    verbose=1 )

def objective(trial): 
    epochs = 10
    
    K.clear_session()   
    model = Sequential()
    
    if trial.suggest_int('Conv', 0, 1):  
        # 1 Convolution layer
        i = 1
        model.add(Conv2D(
            filters = int(trial.suggest_discrete_uniform(
                'Conv_{}_num_filters'.format(i), 32, 64, 32)), 
            kernel_size=tuple([trial.suggest_int(
                'Conv_{}_kernel_size'.format(i), 2, 3)] * 2),
            activation='relu',
            input_shape = input_shape))
        model.add(MaxPooling2D(pool_size=tuple([trial.suggest_int(
                'Conv_{}_max_pooling_size'.format(i), 2, 3)] * 2)))
        model.add(Dropout(trial.suggest_discrete_uniform(
                'Conv_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        model.add(Flatten())        
    else:
        model.add(Flatten(input_shape=input_shape))
    # 2 Fully connected layers
    for i in np.arange(2) + 1:
        model.add(Dense(int(trial.suggest_discrete_uniform(
            'FC_{}_num_hidden_units'.format(i), 256, 512, 256))))
        if trial.suggest_int('FC_{}_batch_normalization'.format(i), 0, 1):
            model.add(BatchNormalization())
        model.add(Activation(trial.suggest_categorical(
            'FC_{}_acivation'.format(i), ['relu'])))
        model.add(Dropout(
            trial.suggest_discrete_uniform(
                'FC_{}_dropout_rate'.format(i), 0, 0.5, 0.25) ))
        
    # Output layer    
    model.add(Dense(num_classes, activation='softmax'))
    
    optimizer_dict = { \
    #'Adagrad': Adagrad(),
    'Adam': Adam() }
    
    model.compile(optimizer = optimizer_dict[
        trial.suggest_categorical('Optimizer', list(optimizer_dict.keys()))],
          loss='sparse_categorical_crossentropy', metrics=['accuracy'])    
    
    if ok.verbose >= 2: model.summary()
    
    batch_size = trial.suggest_int('Batch_size', 256, 256) 
    #batch_size = int(trial.suggest_discrete_uniform(
    #                  'Batch_size', 256, 512, 256) )
    data_augmentation = trial.suggest_int('Data_augmentation', 0, 1)
    
    if not data_augmentation:
        # [Required] Specify callbacks(trial) in fit method
        model.fit(x_train, y_train, batch_size = batch_size,
                  epochs = epochs, validation_data = (x_test, y_test),
                  shuffle = True,
                  callbacks = ok.callbacks(trial), 
                  verbose = ok.keras_verbose )
    
    if data_augmentation:
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            width_shift_range=[-1, 0, +1], # 1 pixel
            height_shift_range=[-1, 0, +1], # 1 pixel
            zoom_range=[0.95,1.05],  # set range for random zoom
            horizontal_flip=False,  # disable horizontal flip
            vertical_flip=False )  # disable vertical flip
        datagen.fit(x_train)
        # [Required] Specify callbacks(trial) in fit_generator method
        model.fit_generator(datagen.flow(x_train, y_train, 
                                         batch_size=batch_size),
                            epochs=epochs, validation_data=(x_test, y_test),
                            steps_per_epoch=len(x_train) // batch_size,
                            callbacks = ok.callbacks(trial), 
                            verbose = ok.keras_verbose )  
    
    # [Required] return trial_best_value (recommended) or latest_value
    return ok.trial_best_value

# Set n_trials and/or timeout (in sec) for optimization by Optuna
ok.optimize(objective, timeout = 3*60) # Run for 3 minutes for demo

[2019-03-08 06:16:12.312888] Ready for optimization. (message printed as verbose is set to 1+)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[2019-03-08 06:16:13.128414] Latest trial id: None, value: None (None) | Best trial id: None, value: None, parameters: None
[2019-03-08 06:18:56.173213] Latest trial id: 1, value: 0.014100000000000001 (TrialState.COMPLETE) | Best trial id: 1, value: 0.014100000000000001, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_augmentation': 1, 'FC_1_acivation': 'relu', 'FC_1_batch_normalization': 1, 'FC_1_dropout_rate': 0.25, 'FC_1_num_hidden_units': 256.0, 'FC_2_acivation': 'relu', 'FC_2_batch_normalization': 1, 'FC_2_dropout_rate': 0.0, 'FC_2_num_hidden_units': 512.0, 'Optimizer': 'Adam'}
[2019-03-08 06:21:35.409784] Latest trial id: 2, value: 0.013599999999999945 (TrialState.COMPLETE) | Best trial id: 2, value: 0.013599999999999945, parameters: {'Batch_size': 256, 'Conv': 0, 'Data_au

In [12]:
""" ## The end of code. """

' ## The end of code. '

Please feel free to post questions or feedback [here](
https://github.com/Minyus/optkeras/issues
)
